In [1]:
import os
import numpy as np
import pandas as pd
import napari
from glob import glob
from tqdm.notebook import tqdm, trange
import sys
sys.path.append(r'../')

from tools import imgLoad
from tools import ioTools as tio
from SegTools import Blocks, Segmentation, Proofread

%gui qt

C:\Users\Yannan\Anaconda3\envs\cellCount38\lib\site-packages\napari\__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)
C:\Users\Yannan\Anaconda3\envs\cellCount38\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
def withinBox(value, x1,x2,y1,y2,z1,z2):
    if len(value.shape) < 2:
        value = value[np.newaxis, :]
    return (value[:,0] > x1) &  (value[:,0] < x2) & (value[:,1] > y1) &  (value[:,1] < y2) & \
            (value[:,2] > z1) &  (value[:,2] < z2) 

def withinMask(mask, coor):
    if mask[int(coor[0]), int(coor[1]), int(coor[2])] > 0:
        return True
    else:
        return False

In [3]:
basedir = r'Y:\People\Yannan\Segmentation\DrugTH10_30mgkg\CKet11823'

imgTH = imgLoad.loadVirtualTifSeq(basedir + r'\data\*.tif')
mask = imgLoad.loadVirtualTifSeq(r"W:\People\Yannan\Segmentation\DF16\regionalMask\aroundHY\highDenseMask2\*.tif") | \
        imgLoad.loadVirtualTifSeq(r"W:\People\Yannan\Segmentation\DF16\regionalMask\aroundHY\aroundHYMask\*.tif")
mask = np.array(mask)
denseRegionBoundBox = tio.loadNpy(basedir + r'\parameters\denseBoundBox.npy')[()]
x1, x2, y1, y2, z1, z2 = denseRegionBoundBox['x1'], denseRegionBoundBox['x2'], denseRegionBoundBox['y1'],\
                        denseRegionBoundBox['y2'], denseRegionBoundBox['z1'], denseRegionBoundBox['z2']

denseRegionCells = Proofread(basedir + r'\regionSpecificTraining', []).cells
# denseRegionCells = pd.read_csv(glob(basedir + r'\regionSpecificTraining\proofreadTemp\manualResults*.csv')[-1])
sparseRegionCells = pd.read_csv(basedir + r"\results.csv")


(5568, 3960, 1140)
(2192, 1776, 750)
(2192, 1776, 750)


In [4]:
sparseCellsWithinBox = withinBox(np.array([sparseRegionCells.x, sparseRegionCells.y, sparseRegionCells.z]).T, \
                                x1, x2, y1, y2, z1, z2)

sparseCellsWithinMask = sparseCellsWithinBox.copy()
for i in tqdm(np.where(sparseCellsWithinBox)[0]):
    coor = [sparseRegionCells.x[i]-x1, sparseRegionCells.y[i]-y1, sparseRegionCells.z[i]-z1]
    sparseCellsWithinMask[i] = mask[int(coor[0]), int(coor[1]), int(coor[2])] > 0
                        

  0%|          | 0/2700 [00:00<?, ?it/s]

In [5]:
denseRegionCellsRemap = denseRegionCells.copy()
denseRegionCellsRemap.x += x1
denseRegionCellsRemap.y += y1
denseRegionCellsRemap.z += z1


In [6]:
fullCells = pd.concat((sparseRegionCells.iloc[~sparseCellsWithinMask], denseRegionCellsRemap))
tio.mkdirs(basedir + r'\fullCells')
fullCells.to_csv(basedir + r'\fullCells\results.csv')

Create  Y:\People\Yannan\Segmentation\DrugTH10_30mgkg\CKet11823\fullCells


In [7]:
%gui qt

viewer = napari.view_points(np.array([fullCells.x, fullCells.y, fullCells.z*2]).T, 
                            size = 10, n_dimensional = True, face_color = 'red', name = 'Cells')
